In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

^C


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train=pd.read_csv(train_file_path,sep='\t',header=None)
df_test=pd.read_csv(test_file_path,sep='\t',header=None)

In [ ]:
df_test

In [ ]:
df_train

In [ ]:
print(df_train.info(),'\t',df_train.describe(),'\n')
print(df_test.info(),'\t',df_test.describe())

In [ ]:
df_train['Length']=df_train[1].apply(len)
df_test['Length']=df_test[1].apply(len)

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x=df_train[0],data=df_train)

In [ ]:
sns.barplot(x=df_train[0],y=df_train["Length"])

In [ ]:
sns.scatterplot(x=df_train[0],y=df_train["Length"])

In [ ]:
df_train[0].value_counts()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import string
from nltk.corpus import stopwords

class PreProcessText(object):
    def __init__(self):
        pass
    
    def __remove_punctuation(self, text):
        message = []
        for x in text:
            if x in string.punctuation:
                pass
            else:
                message.append(x)
        message = ''.join(message)
        return message
    
    def __remove_stopwords(self, text):
        words= []
        for x in text.split():
            if x.lower() in stopwords.words('english'):
                pass
            else:
                words.append(x)
        return words    
    
    def token_words(self,text=''):
        message = self.__remove_punctuation(text)
        words = self.__remove_stopwords(message)
        return words

In [ ]:
mess = 'Sample message! Notice: it has punctuation.'
obj = PreProcessText()
words = obj.token_words(mess)
print(words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
df_train[1]

In [ ]:
bow_transformer = CountVectorizer(analyzer=obj.token_words).fit(df_train[1])

In [ ]:
df_train[1]

In [ ]:
bow_transformer

In [ ]:
messages_bow = bow_transformer.transform(df_train[1])

In [ ]:
messages_bow

In [ ]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bow_transformer = CountVectorizer(analyzer=obj.token_words).fit(df_train[1])
messages_bow = bow_transformer.transform(df_train[1])
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

model = svm.SVC(C=10,kernel='linear')
model.fit(messages_tfidf,df_train[0])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  messages_bow = bow_transformer.transform([pred_text])
  tfidf_transformer = TfidfTransformer().fit(messages_bow)
  messages_tfidf = tfidf_transformer.transform(messages_bow)
  prediction = model.predict(messages_tfidf)


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[0] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
import joblib
joblib.dump(model,'sms_model')